In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import col, lit, udf
from pyspark.sql.functions import sum,avg,max,min,mean,count
spark = SparkSession.builder.appName("Mini Project").getOrCreate()

In [0]:
df = spark.read.options(header='True', inferSchema='True').csv('/FileStore/tables/OfficeDataProject.csv')
df.show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
# total number of employees in a company
df.select('employee_name').count()

Out[3]: 1000

In [0]:
# total number of departments in a company
# df.groupBy("department").count().count()
df.select("department").dropDuplicates(["department"]).count()

Out[5]: 6

In [0]:
# department names in a company
df.select("department").dropDuplicates(["department"]).show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [0]:
# total number of employees in each department
df.groupBy('department').count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|  169|
|        HR|  171|
|   Finance|  162|
|Purchasing|  166|
| Marketing|  170|
|  Accounts|  162|
+----------+-----+



In [0]:
#  total number of employees in each state
df.groupBy('state').count().show()

+-----+-----+
|state|count|
+-----+-----+
|   LA|  205|
|   CA|  205|
|   WA|  208|
|   NY|  173|
|   AK|  209|
+-----+-----+



In [0]:
# total number of employees in each state in each department
df.groupBy('state','department').count().show()

+-----+----------+-----+
|state|department|count|
+-----+----------+-----+
|   CA|     Sales|   42|
|   CA| Marketing|   33|
|   NY|  Accounts|   34|
|   NY|     Sales|   27|
|   CA|   Finance|   35|
|   CA|  Accounts|   35|
|   CA|Purchasing|   32|
|   WA|        HR|   47|
|   AK|Purchasing|   30|
|   WA|  Accounts|   27|
|   WA|Purchasing|   38|
|   AK|     Sales|   38|
|   AK|  Accounts|   37|
|   WA| Marketing|   39|
|   LA|        HR|   41|
|   LA|     Sales|   35|
|   AK|        HR|   25|
|   LA|   Finance|   29|
|   AK|   Finance|   37|
|   LA|Purchasing|   45|
+-----+----------+-----+
only showing top 20 rows



In [0]:
df.show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
#min and max salaries in each department, sorting in asc order
df.groupBy("department").agg(min("salary").alias("min"), max("salary").alias("max")).orderBy(col("max").asc(), col("min").asc()).show()

+----------+----+----+
|department| min| max|
+----------+----+----+
|  Accounts|1007|9890|
|   Finance|1006|9899|
| Marketing|1031|9974|
|        HR|1013|9982|
|     Sales|1103|9982|
|Purchasing|1105|9985|
+----------+----+----+



In [0]:
#names of employees working in a NY state under Finance department whose bonuses are greater than the average bonuses of employees in NY state

In [0]:
avgBonus = df.filter((df.state =='NY')).groupBy("state", "department").agg(avg("bonus").alias('avg_bonus')).select("avg_bonus").collect()[0]['avg_bonus']
df.filter((df.state =='NY') & (df.department == 'Finance') & (df.bonus >= avgBonus)).show()

+-----------+--------------------+----------+-----+------+---+-----+
|employee_id|       employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
|       1035|       Vivan Sifford|   Finance|   NY|  1129| 35| 1261|
|       1073|      Herder Gallman|   Finance|   NY|  1988| 31| 1402|
|       1082|          Nena Rocha|   Finance|   NY|  3417| 25| 1647|
|       1087|       Leif Lemaster|   Finance|   NY|  8642| 45| 1782|
|       1100|Ellingsworth Meli...|   Finance|   NY|  7845| 32| 1358|
|       1127|        Escoto Gilma|   Finance|   NY|  3426| 41| 1285|
|       1161|     Georgeanna Laub|   Finance|   NY|  2469| 26| 1679|
|       1175|     Durio Tenenbaum|   Finance|   NY|  2253| 42| 1684|
|       1180|       Juliana Grigg|   Finance|   NY|  8178| 42| 1617|
|       1215|        Tiffani Benz|   Finance|   NY|  1665| 41| 1969|
|       1220|          Nitz Ilana|   Finance|   NY|  2443| 50| 1342|
|       1342|   Phylicia Antonina|

In [0]:
#raise the salaries $500 of all employees whose age is greater than 45

In [0]:
def get_raise(age, salary):
    sum=0
    if age > 45:
        sum = salary+500
    return sum

In [0]:
newSalaryUDF = udf(lambda x,y: get_raise(x,y), DoubleType())

df.withColumn("total_salary", newSalaryUDF(df.age, df.salary)).show()
     

+-----------+-------------------+----------+-----+------+---+-----+------------+
|employee_id|      employee_name|department|state|salary|age|bonus|total_salary|
+-----------+-------------------+----------+-----+------+---+-----+------------+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|        null|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|        null|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|        null|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|        null|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|        null|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|        null|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|        null|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|        null|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|        null|
|       1009|        Imai Ha

In [0]:
#create DF of all employees whose age is greater than 45 and save them in file

In [0]:
df45 = df.filter(df.age>45).write.mode("overwrite").options(header='True').csv('/FileStore/tables/StudentData/output_45')